In [ ]:
import pandas as pd
import os
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 htt

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [ ]:
from pyspark import SparkFiles
url = "http://mcr-bootcamp-final-project.s3.us-east-2.amazonaws.com/mbti_1.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("mbti_1.csv"), sep=",", header=True)

In [ ]:
df.show(10)

+---+---+---+---+--------------------+
|E/I|S/N|T/F|J/P|               posts|
+---+---+---+---+--------------------+
|  I|  N|  F|  J|'http://www.youtu...|
|  E|  N|  T|  P|'I'm finding the ...|
|  I|  N|  T|  P|'Good one  _____ ...|
|  I|  N|  T|  J|'Dear INTP,   I e...|
|  E|  N|  T|  J|'You're fired.|||...|
|  I|  N|  T|  J|'18/37 @.@|||Scie...|
|  I|  N|  F|  J|'No, I can't draw...|
|  I|  N|  T|  J|'I tend to build ...|
|  I|  N|  F|  J|I'm not sure, tha...|
|  I|  N|  T|  P|'https://www.yout...|
+---+---+---+---+--------------------+
only showing top 10 rows



In [ ]:
pandasDF = df.toPandas()
pandasDF

,E/I,S/N,T/F,J/P,posts
0,I,N,F,J,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,E,N,T,P,'I'm finding the lack of me in these posts ver...
2,I,N,T,P,'Good one _____ https://www.youtube.com/wat...
3,I,N,T,J,"'Dear INTP, I enjoyed our conversation the o..."
4,E,N,T,J,'You're fired.|||That's another silly misconce...
...,...,...,...,...,...
8670,I,S,F,P,'https://www.youtube.com/watch?v=t8edHB_h908||...
8671,E,N,F,P,'So...if this thread already exists someplace ...
8672,I,N,T,P,'So many questions when i do these things. I ...
8673,I,N,F,P,'I am very conflicted right now when it comes ...


In [ ]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['posts']))
data_df.show()

+---+---+---+---+--------------------+------+
|E/I|S/N|T/F|J/P|               posts|length|
+---+---+---+---+--------------------+------+
|  I|  N|  F|  J|'http://www.youtu...|  4652|
|  E|  N|  T|  P|'I'm finding the ...|  7053|
|  I|  N|  T|  P|'Good one  _____ ...|  5265|
|  I|  N|  T|  J|'Dear INTP,   I e...|  6271|
|  E|  N|  T|  J|'You're fired.|||...|  6111|
|  I|  N|  T|  J|'18/37 @.@|||Scie...|  8589|
|  I|  N|  F|  J|'No, I can't draw...|  7916|
|  I|  N|  T|  J|'I tend to build ...|  6900|
|  I|  N|  F|  J|I'm not sure, tha...|  5325|
|  I|  N|  T|  P|'https://www.yout...|  7573|
|  I|  N|  F|  J|'One time my pare...|  8381|
|  E|  N|  F|  J|'https://www.yout...|  4724|
|  I|  N|  F|  J|'Joe santagato - ...|  7007|
|  I|  N|  T|  J|'Fair enough, if ...|  9035|
|  I|  N|  T|  P|'Basically this.....|  5291|
|  I|  N|  T|  P|'Your comment scr...|  7244|
|  I|  N|  F|  J|'some of these bo...|  8253|
|  I|  N|  F|  P|'I think we do ag...|  9489|
|  I|  N|  F|  J|'I fully believe 

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num_ei = StringIndexer(inputCol='E/I',outputCol='label')
pos_neg_to_num_sn = StringIndexer(inputCol='S/N',outputCol='label')
pos_neg_to_num_tf = StringIndexer(inputCol='T/F',outputCol='label')
pos_neg_to_num_jp = StringIndexer(inputCol='J/P',outputCol='label')
tokenizer = Tokenizer(inputCol="posts", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline_ei = Pipeline(stages=[pos_neg_to_num_ei, tokenizer, stopremove, hashingTF, idf, clean_up])
data_prep_pipeline_sn = Pipeline(stages=[pos_neg_to_num_sn, tokenizer, stopremove, hashingTF, idf, clean_up])
data_prep_pipeline_tf = Pipeline(stages=[pos_neg_to_num_tf, tokenizer, stopremove, hashingTF, idf, clean_up])
data_prep_pipeline_jp = Pipeline(stages=[pos_neg_to_num_jp, tokenizer, stopremove, hashingTF, idf, clean_up])


In [ ]:
# Fit and transform the pipeline
cleaner_ei = data_prep_pipeline_ei.fit(data_df)
cleaner_sn = data_prep_pipeline_sn.fit(data_df)
cleaner_tf = data_prep_pipeline_tf.fit(data_df)
cleaner_jp = data_prep_pipeline_jp.fit(data_df)
cleaned_ei = cleaner_ei.transform(data_df)
cleaned_sn = cleaner_sn.transform(data_df)
cleaned_tf = cleaner_tf.transform(data_df)
cleaned_jp = cleaner_jp.transform(data_df)

In [ ]:
cleaned_ei.select(['label', 'features']).show()
cleaned_sn.select(['label', 'features']).show()
cleaned_tf.select(['label', 'features']).show()
cleaned_jp.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[619,696,...|
|  1.0|(262145,[23,619,9...|
|  0.0|(262145,[142,991,...|
|  0.0|(262145,[329,462,...|
|  1.0|(262145,[2120,243...|
|  0.0|(262145,[432,896,...|
|  0.0|(262145,[134,356,...|
|  0.0|(262145,[303,1042...|
|  0.0|(262145,[619,826,...|
|  0.0|(262145,[107,378,...|
|  0.0|(262145,[198,991,...|
|  1.0|(262145,[1546,161...|
|  0.0|(262145,[319,522,...|
|  0.0|(262145,[462,1061...|
|  0.0|(262145,[141,415,...|
|  0.0|(262145,[991,1510...|
|  0.0|(262145,[329,438,...|
|  0.0|(262145,[991,1164...|
|  0.0|(262145,[735,2021...|
|  0.0|(262145,[329,751,...|
+-----+--------------------+
only showing top 20 rows

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[619,696,...|
|  0.0|(262145,[23,619,9...|
|  0.0|(262145,[142,991,...|
|  0.0|(262145,[329,462,...|
|  0.0|(262145,[2120,243...|
|  0.0|(262145,[432,896,...|
|  0.0|(262145,[1

In [ ]:
from pyspark.ml.classification import NaiveBayes
training_ei, testing_ei = cleaned_ei.randomSplit([0.9, 0.1])
training_sn, testing_sn = cleaned_sn.randomSplit([0.9, 0.1])
training_tf, testing_tf = cleaned_tf.randomSplit([0.9, 0.1])
training_jp, testing_jp = cleaned_jp.randomSplit([0.8, 0.1])

nb = NaiveBayes()

predictor_ei = nb.fit(training_ei)
predictor_sn = nb.fit(training_sn)
predictor_tf = nb.fit(training_tf)
predictor_jp = nb.fit(training_jp)

In [ ]:
test_results_ei = predictor_ei.transform(testing_ei)
test_results_sn = predictor_sn.transform(testing_sn)
test_results_tf = predictor_tf.transform(testing_tf)
test_results_jp = predictor_jp.transform(testing_jp)

test_results_ei.show(5)
test_results_sn.show(5)
test_results_tf.show(5)
test_results_jp.show(5)

+---+---+---+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|E/I|S/N|T/F|J/P|               posts|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+---+---+---+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  E|  N|  F|  J|'Actually, there ...|  5265|  1.0|['actually,, ther...|['actually,, movi...|(262144,[329,1892...|(262144,[329,1892...|(262145,[329,1892...|[-16868.799491911...|[1.0,3.2764880734...|       0.0|
|  E|  N|  F|  J|'Have any of you ...|  5545|  1.0|['have, any, of, ...|['have, ever, dep...|(262144,[329,991,...|(262144,[329,991,...|(262145,[329,991,...|[-17

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()

acc_ei = acc_eval.evaluate(test_results_ei)
acc_sn = acc_eval.evaluate(test_results_sn)
acc_tf = acc_eval.evaluate(test_results_tf)
acc_jp = acc_eval.evaluate(test_results_jp)

print("Accuracy of model at predicting E/I was: %f" % acc_ei)
print("Accuracy of model at predicting S/N was: %f" % acc_sn)
print("Accuracy of model at predicting T/F was: %f" % acc_tf)
print("Accuracy of model at predicting J/P was: %f" % acc_jp)

Accuracy of model at predicting E/I was: 0.671097
Accuracy of model at predicting S/N was: 0.816498
Accuracy of model at predicting T/F was: 0.699264
Accuracy of model at predicting J/P was: 0.591003


In [ ]:
from pyspark.sql.functions import col, concat_ws
  
df2 = test_results_ei.withColumn("token_text",
      concat_ws(",",col("token_text")))
df2.printSchema()
df2.show(5)

root
 |-- E/I: string (nullable = true)
 |-- S/N: string (nullable = true)
 |-- T/F: string (nullable = true)
 |-- J/P: string (nullable = true)
 |-- posts: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- token_text: string (nullable = false)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- hash_token: vector (nullable = true)
 |-- idf_token: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)

+---+---+---+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|E/I|S/N|T/F|J/P|               posts|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|        

In [ ]:
import pyspark.sql.functions as F
df2.withColumn('E/I', F.length('token_text')).show(5)

+----+---+---+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| E/I|S/N|T/F|J/P|               posts|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+----+---+---+---+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|5265|  N|  F|  J|'Actually, there ...|  5265|  1.0|'actually,,there,...|['actually,, movi...|(262144,[329,1892...|(262144,[329,1892...|(262145,[329,1892...|[-16868.799491911...|[1.0,3.2764880734...|       0.0|
|5545|  N|  F|  J|'Have any of you ...|  5545|  1.0|'have,any,of,you,...|['have, ever, dep...|(262144,[329,991,...|(262144,[329,991,...|(262145,[329,991,...

In [ ]:
pandasDF = test_results_ei.toPandas()

In [ ]:
pandasDF.head()

,E/I,S/N,T/F,J/P,posts,length,label,token_text,stop_tokens,hash_token,idf_token,features,rawPrediction,probability,prediction
0,E,N,F,J,"'Actually, there are a few movies and shows th...",5265,1.0,"['actually,, there, are, a, few, movies, and, ...","['actually,, movies, shows, enjoy, sources,, l...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-16868.79949191187, -16980.4393893291]","[1.0, 3.276488073487599e-49]",0.0
1,E,N,F,J,'Have any of you ever been depressed? How woul...,5545,1.0,"['have, any, of, you, ever, been, depressed?, ...","['have, ever, depressed?, one, help, entj, cop...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-17610.203137202378, -17526.858334873206]","[6.3652029168827365e-37, 1.0]",1.0
2,E,N,F,J,'I have felt sad in the past when people don't...,6029,1.0,"['i, have, felt, sad, in, the, past, when, peo...","['i, felt, sad, past, people, acknowledge, int...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-19549.49486753164, -19772.83954047228]","[1.0, 1.006099607085692e-97]",0.0
3,E,N,F,J,'I have the same feeling quite often :D|||Than...,8102,1.0,"['i, have, the, same, feeling, quite, often, :...","['i, feeling, quite, often, :d|||thank, well, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-23559.696788554993, -23723.03257027056]","[1.0, 1.1592345156780305e-71]",0.0
4,E,N,F,J,'I'm currently dating an ISFP and it's the mos...,9604,1.0,"['i'm, currently, dating, an, isfp, and, it's,...","['i'm, currently, dating, isfp, magical, relat...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-30965.29591220575, -31275.384325678573]","[1.0, 2.1395041174701995e-135]",0.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pandasDF.to_csv('/content/drive/MyDrive/pandasDF.csv', index=False, encoding='utf-8', header='true')  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pandasDF['stop_tokens_2'] = pandasDF['stop_tokens'].apply(set)

In [ ]:
pandasDF.head()

,E/I,S/N,T/F,J/P,posts,length,label,token_text,stop_tokens,hash_token,idf_token,features,rawPrediction,probability,prediction,stop_tokens_2
0,E,N,F,J,"'Actually, there are a few movies and shows th...",5265,1.0,"['actually,, there, are, a, few, movies, and, ...","['actually,, movies, shows, enjoy, sources,, l...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-16868.79949191187, -16980.4393893291]","[1.0, 3.276488073487599e-49]",0.0,"{, decisive, :happy:|||no., anything?|||here, ..."
1,E,N,F,J,'Have any of you ever been depressed? How woul...,5545,1.0,"['have, any, of, you, ever, been, depressed?, ...","['have, ever, depressed?, one, help, entj, cop...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-17610.203137202378, -17526.858334873206]","[6.3652029168827365e-37, 1.0]",1.0,"{, too,, depressed?, :p|||quickly, perfect, (e..."
2,E,N,F,J,'I have felt sad in the past when people don't...,6029,1.0,"['i, have, felt, sad, in, the, past, when, peo...","['i, felt, sad, past, people, acknowledge, int...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-19549.49486753164, -19772.83954047228]","[1.0, 1.006099607085692e-97]",0.0,"{, mark, happen., character, referring, third,..."
3,E,N,F,J,'I have the same feeling quite often :D|||Than...,8102,1.0,"['i, have, the, same, feeling, quite, often, :...","['i, feeling, quite, often, :d|||thank, well, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-23559.696788554993, -23723.03257027056]","[1.0, 1.1592345156780305e-71]",0.0,"{, starting, something, gets, uncomfortable, g..."
4,E,N,F,J,'I'm currently dating an ISFP and it's the mos...,9604,1.0,"['i'm, currently, dating, an, isfp, and, it's,...","['i'm, currently, dating, isfp, magical, relat...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-30965.29591220575, -31275.384325678573]","[1.0, 2.1395041174701995e-135]",0.0,"{, something, gets, within, matter, goodness!,..."


In [ ]:
print(len(pandasDF['stop_tokens_2'][0]))

363


In [ ]:
import numpy as np
pandasDF['unique_words'] = np.nan

In [ ]:
pandasDF.head()

,E/I,S/N,T/F,J/P,posts,length,label,token_text,stop_tokens,hash_token,idf_token,features,rawPrediction,probability,prediction,stop_tokens_2,unique_words
0,E,N,F,J,"'Actually, there are a few movies and shows th...",5265,1.0,"['actually,, there, are, a, few, movies, and, ...","['actually,, movies, shows, enjoy, sources,, l...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-16868.79949191187, -16980.4393893291]","[1.0, 3.276488073487599e-49]",0.0,"{, decisive, :happy:|||no., anything?|||here, ...",NaN
1,E,N,F,J,'Have any of you ever been depressed? How woul...,5545,1.0,"['have, any, of, you, ever, been, depressed?, ...","['have, ever, depressed?, one, help, entj, cop...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-17610.203137202378, -17526.858334873206]","[6.3652029168827365e-37, 1.0]",1.0,"{, too,, depressed?, :p|||quickly, perfect, (e...",NaN
2,E,N,F,J,'I have felt sad in the past when people don't...,6029,1.0,"['i, have, felt, sad, in, the, past, when, peo...","['i, felt, sad, past, people, acknowledge, int...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-19549.49486753164, -19772.83954047228]","[1.0, 1.006099607085692e-97]",0.0,"{, mark, happen., character, referring, third,...",NaN
3,E,N,F,J,'I have the same feeling quite often :D|||Than...,8102,1.0,"['i, have, the, same, feeling, quite, often, :...","['i, feeling, quite, often, :d|||thank, well, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-23559.696788554993, -23723.03257027056]","[1.0, 1.1592345156780305e-71]",0.0,"{, starting, something, gets, uncomfortable, g...",NaN
4,E,N,F,J,'I'm currently dating an ISFP and it's the mos...,9604,1.0,"['i'm, currently, dating, an, isfp, and, it's,...","['i'm, currently, dating, isfp, magical, relat...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-30965.29591220575, -31275.384325678573]","[1.0, 2.1395041174701995e-135]",0.0,"{, something, gets, within, matter, goodness!,...",NaN


In [ ]:
pandasDF['unique_words'] = pandasDF['stop_tokens_2'].apply(lambda x: len(x))

In [ ]:
pandasDF.head()

,E/I,S/N,T/F,J/P,posts,length,label,token_text,stop_tokens,hash_token,idf_token,features,rawPrediction,probability,prediction,stop_tokens_2,unique_words
0,E,N,F,J,"'Actually, there are a few movies and shows th...",5265,1.0,"['actually,, there, are, a, few, movies, and, ...","['actually,, movies, shows, enjoy, sources,, l...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-16868.79949191187, -16980.4393893291]","[1.0, 3.276488073487599e-49]",0.0,"{, decisive, :happy:|||no., anything?|||here, ...",363
1,E,N,F,J,'Have any of you ever been depressed? How woul...,5545,1.0,"['have, any, of, you, ever, been, depressed?, ...","['have, ever, depressed?, one, help, entj, cop...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-17610.203137202378, -17526.858334873206]","[6.3652029168827365e-37, 1.0]",1.0,"{, too,, depressed?, :p|||quickly, perfect, (e...",349
2,E,N,F,J,'I have felt sad in the past when people don't...,6029,1.0,"['i, have, felt, sad, in, the, past, when, peo...","['i, felt, sad, past, people, acknowledge, int...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-19549.49486753164, -19772.83954047228]","[1.0, 1.006099607085692e-97]",0.0,"{, mark, happen., character, referring, third,...",378
3,E,N,F,J,'I have the same feeling quite often :D|||Than...,8102,1.0,"['i, have, the, same, feeling, quite, often, :...","['i, feeling, quite, often, :d|||thank, well, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-23559.696788554993, -23723.03257027056]","[1.0, 1.1592345156780305e-71]",0.0,"{, starting, something, gets, uncomfortable, g...",510
4,E,N,F,J,'I'm currently dating an ISFP and it's the mos...,9604,1.0,"['i'm, currently, dating, an, isfp, and, it's,...","['i'm, currently, dating, isfp, magical, relat...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-30965.29591220575, -31275.384325678573]","[1.0, 2.1395041174701995e-135]",0.0,"{, something, gets, within, matter, goodness!,...",632


In [ ]:
pandasDF = pandasDF[['E/I', 'S/N', 'T/F', 'J/P', 'posts', 'unique_words']]
pandasDF.head()

,E/I,S/N,T/F,J/P,posts,unique_words
0,E,N,F,J,"'Actually, there are a few movies and shows th...",363
1,E,N,F,J,'Have any of you ever been depressed? How woul...,349
2,E,N,F,J,'I have felt sad in the past when people don't...,378
3,E,N,F,J,'I have the same feeling quite often :D|||Than...,510
4,E,N,F,J,'I'm currently dating an ISFP and it's the mos...,632


In [ ]:
pandasDF.to_csv('/content/drive/MyDrive/pandasDF2.csv', index=False, encoding='utf-8', header='true')

In [ ]:
#url2 = "/content/drive/MyDrive/pandasDF2.csv"
#spark.sparkContext.addFile(url)
model_2_df = spark.read.csv(SparkFiles.get("/content/drive/MyDrive/pandasDF2.csv"), sep=",", header=True)

In [ ]:
from pyspark.sql.types import IntegerType

model_2_df = model_2_df.withColumn("unique_words", model_2_df["unique_words"].cast(IntegerType()))
model_2_df.printSchema()
model_2_df.show()

root
 |-- E/I: string (nullable = true)
 |-- S/N: string (nullable = true)
 |-- T/F: string (nullable = true)
 |-- J/P: string (nullable = true)
 |-- posts: string (nullable = true)
 |-- unique_words: integer (nullable = true)

+---+---+---+---+--------------------+------------+
|E/I|S/N|T/F|J/P|               posts|unique_words|
+---+---+---+---+--------------------+------------+
|  E|  N|  F|  J|'Actually, there ...|         363|
|  E|  N|  F|  J|'Have any of you ...|         349|
|  E|  N|  F|  J|'I have felt sad ...|         378|
|  E|  N|  F|  J|'I have the same ...|         510|
|  E|  N|  F|  J|'I'm currently da...|         632|
|  E|  N|  F|  J|'Sorry I know thi...|         498|
|  E|  N|  F|  J|'When I get angry...|         364|
|  E|  N|  F|  J|'YAYY!!! That was...|         512|
|  E|  N|  F|  J|'You described ex...|         560|
|  E|  N|  F|  J|'You found out so...|         595|
|  E|  N|  F|  J|'amc  Hello amc! ...|         600|
|  E|  N|  F|  J|You are a Benevol...|      

In [ ]:
pos_neg_to_num_ei2 = StringIndexer(inputCol='E/I',outputCol='label')
pos_neg_to_num_sn2 = StringIndexer(inputCol='S/N',outputCol='label')
pos_neg_to_num_tf2 = StringIndexer(inputCol='T/F',outputCol='label')
pos_neg_to_num_jp2 = StringIndexer(inputCol='J/P',outputCol='label')
tokenizer2 = Tokenizer(inputCol="posts", outputCol="token_text")
stopremove2 = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF2 = HashingTF(inputCol="stop_tokens", outputCol='hash_token')
idf2 = IDF(inputCol='hash_token', outputCol='idf_token')

In [ ]:
clean_up2 = VectorAssembler(inputCols=['idf_token', 'unique_words'], outputCol='features')

In [ ]:
data_prep_pipeline_ei2 = Pipeline(stages=[pos_neg_to_num_ei2, tokenizer2, stopremove2, hashingTF2, idf2, clean_up2])
data_prep_pipeline_sn2 = Pipeline(stages=[pos_neg_to_num_sn2, tokenizer2, stopremove2, hashingTF2, idf2, clean_up2])
data_prep_pipeline_tf2 = Pipeline(stages=[pos_neg_to_num_tf2, tokenizer2, stopremove2, hashingTF2, idf2, clean_up2])
data_prep_pipeline_jp2 = Pipeline(stages=[pos_neg_to_num_jp2, tokenizer2
, stopremove2, hashingTF2, idf2, clean_up2])

In [ ]:
cleaner_ei2 = data_prep_pipeline_ei2.fit(model_2_df)
cleaner_sn2 = data_prep_pipeline_sn2.fit(model_2_df)
cleaner_tf2 = data_prep_pipeline_tf2.fit(model_2_df)
cleaner_jp2 = data_prep_pipeline_jp2.fit(model_2_df)
cleaned_ei2 = cleaner_ei2.transform(model_2_df)
cleaned_sn2 = cleaner_sn2.transform(model_2_df)
cleaned_tf2 = cleaner_tf2.transform(model_2_df)
cleaned_jp2 = cleaner_jp2.transform(model_2_df)

In [ ]:
training_ei2, testing_ei2 = cleaned_ei2.randomSplit([0.9, 0.1])
training_sn2, testing_sn2 = cleaned_sn2.randomSplit([0.9, 0.1])
training_tf2, testing_tf2 = cleaned_tf2.randomSplit([0.9, 0.1])
training_jp2, testing_jp2 = cleaned_jp2.randomSplit([0.9, 0.1])

predictor_ei2 = nb.fit(training_ei2)
predictor_sn2 = nb.fit(training_sn2)
predictor_tf2 = nb.fit(training_tf2)
predictor_jp2 = nb.fit(training_jp2)

In [ ]:
test_results_ei2 = predictor_ei2.transform(testing_ei2)
test_results_sn2 = predictor_sn2.transform(testing_sn2)
test_results_tf2 = predictor_tf2.transform(testing_tf2)
test_results_jp2 = predictor_jp2.transform(testing_jp2)

test_results_ei2.show(5)
test_results_sn2.show(5)
test_results_tf2.show(5)
test_results_jp2.show(5)

+---+---+---+---+--------------------+------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|E/I|S/N|T/F|J/P|               posts|unique_words|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+---+---+---+---+--------------------+------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  E|  N|  F|  P|'...now I have so...|         528|  1.0|['...now, i, have...|['...now, somethi...|(262144,[216,1782...|(262144,[216,1782...|(262145,[216,1782...|[-21225.557472836...|[1.0,1.3647478398...|       0.0|
|  E|  N|  F|  P|':shocked: Psssh....|         447|  1.0|[':shocked:, psss...|[':shocked:, psss...|(262144,[560,991,...|(262144,[560,991

In [ ]:
acc_eval2 = MulticlassClassificationEvaluator()

acc_ei2 = acc_eval2.evaluate(test_results_ei2)
acc_sn2 = acc_eval2.evaluate(test_results_sn2)
acc_tf2 = acc_eval2.evaluate(test_results_tf2)
acc_jp2 = acc_eval2.evaluate(test_results_jp2)

print("Model using vocab size to evaluate personality type:")
print("Accuracy of model at predicting E/I was: %f" % acc_ei2)
print("Accuracy of model at predicting S/N was: %f" % acc_sn2)
print("Accuracy of model at predicting T/F was: %f" % acc_tf2)
print("Accuracy of model at predicting J/P was: %f" % acc_jp2)

Model using vocab size to evaluate personality type:
Accuracy of model at predicting E/I was: 0.732143
Accuracy of model at predicting S/N was: 0.863242
Accuracy of model at predicting T/F was: 0.581878
Accuracy of model at predicting J/P was: 0.552376
